This notebook is for newbies from a newbie, it will help you to get started with the problem
1. First I will describe about initial EDA.
2. Then we will make simple model using pretrained weights from EfficientNet network

Hope you like the efforts.
PLease upvote if this notebook helped you in any way

### Importing the Libraries 

In [ ]:
import os
import random

import numpy as np 
import pandas as pd 
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout, Input, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from glob import glob

import seaborn as sns
from pathlib import Path

### Reading the Path
Note here we are taking a new dataset in which all the duplicate and bad noisy images has been removed

In [ ]:
#source path (where the Pawpularity contest data resides)
path = '../input/petfinder-pawpularity-score-clean/'
path_test = '../input/petfinder-pawpularity-score/'
#Get the metadata (the .csv data) and put it into DataFrames
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')

#Get the image data (the .jpg data) and put it into lists of filenames
train_jpg = glob(path + "train/*.jpg")
test_jpg = glob(path + "test/*.jpg")

### Getting the dimensions for our data

In [ ]:
#show the dimensions of the train metadata.
print('train_df dimensions: ', train_df.shape)
print('train_df column names: ', train_df.columns.values.tolist())

#print an extra row could use '\n' as well in a print statement
print('')

#show the dimensions of the test metadata
print('test_df dimensions: ',test_df.shape)
print('test_df column names: ', test_df.columns.values.tolist())

In [ ]:
#show the type of train_jpg and test_jpg as well as length of the list.
print('train_jpg is of type ',type(train_jpg), ' and length ', len(train_jpg))
#Also show the first 3 elements
print('train_jpg list 1st 3 elements: ', train_jpg[0:3], '\n')

print('test_jpg is of type ',type(test_jpg), ' and length ', len(test_jpg))
#Also show the first 3 elements
print('test_jpg list 1st 3 elements: ', test_jpg[0:3])

### Distribution of Target Variable



In [ ]:

sns.set(rc={'figure.figsize':(15,5)})
fig = plt.figure()
sns.histplot(data=train_df, x='Pawpularity', bins=100)
plt.axvline(train_df['Pawpularity'].mean(), c='red', ls='-', lw=3, label='Mean Pawpularity')
plt.axvline(train_df['Pawpularity'].median(),c='blue',ls='-',lw=3, label='Median Pawpularity')
plt.title('Distribution of Pawpularity Scores', fontsize=20, fontweight='bold')
plt.legend()
plt.show()

### lets visualize the distribution of Pawpularity scores across each feature variable


In [ ]:
#Let's start with just one variable to demonstrate
fig, ax = plt.subplots(1,2)
sns.boxplot(data=train_df, x='Eyes', y='Pawpularity', ax=ax[0])
sns.histplot(train_df, x="Pawpularity", hue="Eyes", kde=True, ax=ax[1])
plt.suptitle("Eyes", fontsize=20, fontweight='bold')
fig.show()

In [ ]:
#Let's start with just one variable to demonstrate
fig, ax = plt.subplots(1,2)
sns.boxplot(data=train_df, x='Face', y='Pawpularity', ax=ax[0])
sns.histplot(train_df, x="Pawpularity", hue="Eyes", kde=True, ax=ax[1])
plt.suptitle("Eyes", fontsize=20, fontweight='bold')
fig.show()

In [ ]:
#Let's start with just one variable to demonstrate
fig, ax = plt.subplots(1,2)
sns.boxplot(data=train_df, x='Near', y='Pawpularity', ax=ax[0])
sns.histplot(train_df, x="Pawpularity", hue="Near", kde=True, ax=ax[1])
plt.suptitle("Eyes", fontsize=20, fontweight='bold')
fig.show()

In [ ]:
#Let's start with just one variable to demonstrate
fig, ax = plt.subplots(1,2)
sns.boxplot(data=train_df, x='Action', y='Pawpularity', ax=ax[0])
sns.histplot(train_df, x="Pawpularity", hue="Action", kde=True, ax=ax[1])
plt.suptitle("Eyes", fontsize=20, fontweight='bold')
fig.show()

### Unable to find anything sigficant from individual features

### Now we will perform EDA on Image data

### Sample Training Images

In [ ]:
#first let's try printing out the first 3 images with a loop

#The for loop goes for 3 loops 
for x in range(3):
    #this loop goes through index of the train_jpg list of filenames: 0,1,2
    image_path = train_jpg[x]
    #use plt.imread() to read in that image file as an array of numbers between 0-255
    image_array = plt.imread(image_path) 
    #Let's check the image dimensions
    print("image {}'s dimensions are: {}".format(x,image_array.shape))
    #then plt.imshow() can display it for you
    plt.imshow(image_array)
    #title is the index of train_jpg
    plt.title(x) 
    #turn off gridlines
    plt.axis('off')
    #show the image
    plt.show()

### Sample Testing Images


**NOTE** the test images are just random noices, so nothing to worry final scoring will be done on actual images, thats why in this competition we need to submit notebook also.

In [ ]:
#first let's try printing out the first 3 images with a loop

#The for loop goes for 3 loops 
for x in range(8):
    #this loop goes through index of the train_jpg list of filenames: 0,1,2
    image_path = test_jpg[x]
    #use plt.imread() to read in that image file as an array of numbers between 0-255
    image_array = plt.imread(image_path) 
    #Let's check the image dimensions
    print("image {}'s dimensions are: {}".format(x,image_array.shape))
    #then plt.imshow() can display it for you
    plt.imshow(image_array)
    #title is the index of train_jpg
    plt.title(x) 
    #turn off gridlines
    plt.axis('off')
    #show the image
    plt.show()

### Lets MAKE OUR FIRST MODEL USING PRETRAINED WEIGHTS

### Setting up the hyper parameters

In [ ]:

AUTOTUNE = tf.data.experimental.AUTOTUNE  
img_size = 224
channels = 3
Batch_size = 128



def seed_everything():
    np.random.seed(123)
    random.seed(123)
    tf.random.set_seed(123)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
    os.environ['PYTHONHASHSEED'] = str(123)

seed_everything()

In [ ]:
### reading the data
df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score-clean/train.csv")
df_test = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
Id = df_test["Id"].copy()


df["Id"] = df["Id"].apply(lambda x : "/kaggle/input/petfinder-pawpularity-score-clean/train/" + x + ".jpg")
df_test["Id"] = df_test["Id"].apply(lambda x : "/kaggle/input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [ ]:
### doing data augmentation in images
def image_preprocess(is_labelled):  
    def augment(image):
        image = tf.image.random_flip_left_right(image)

        image = tf.image.random_saturation(image, 0.95, 1.05)
        image = tf.image.random_contrast(image, 0.95, 1.05)
        return image
    
    def can_be_augmented(img, label):
        return augment(img), label
    

    return can_be_augmented if is_labelled else augment




def image_read(is_labelled):
    def decode(path):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=channels)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (img_size, img_size))
        image = tf.keras.applications.efficientnet.preprocess_input(image) 
        return image
    
    def can_be_decoded(path, label):
        return decode(path), label
    
#   If record has label both image and lable will be returned

    return can_be_decoded if is_labelled else decode



def create_dataset(df, batch_size, is_labelled = False, augment = False, shuffle = False):
    image_read_fn = image_read(is_labelled)
    image_preprocess_fn = image_preprocess(is_labelled)
    
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df["Id"].values, df["Pawpularity"].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df["Id"].values))
    
    dataset = dataset.map(image_read_fn, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(image_preprocess_fn, num_parallel_calls=AUTOTUNE) if augment else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


In [ ]:
df

### Creating train test validation set

In [ ]:
trn = df.iloc[:9000]
val = df.iloc[9001:]



train = create_dataset(trn, Batch_size, is_labelled = True, augment = True, shuffle = True)
validation = create_dataset(val, Batch_size, is_labelled = True, augment = False, shuffle = False)
test = create_dataset(df_test, Batch_size, is_labelled = False, augment = False, shuffle=False)

In [ ]:
print(trn.shape)
print(val.shape)
print(df_test.shape)

### Extracting Pretrained efficientNet Weights

In [ ]:
img_mod = "/kaggle/input/keras-applications-models/EfficientNetB0.h5"
efnet = tf.keras.models.load_model(img_mod)

efnet.trainable = False

### Adding additional Layers for prediction

In [ ]:
model = Sequential([
    Input(shape=(img_size, img_size, channels)),
    efnet,
    BatchNormalization(),
    Dropout(0.2),
    Dense(units = 128, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),
    Dense(units = 64, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),
    Dense(units = 1, activation="relu")
])


### addding early stoping and other hyperparameters

In [ ]:
early_stopping = EarlyStopping(patience = 5,restore_best_weights=True)


lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100, decay_rate=0.96,
    staircase=True)

### Compiling the Model

In [ ]:
model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train,
                      epochs=50, 
                      validation_data = validation,
                      callbacks=[early_stopping])

### Making Final Prediction

In [ ]:
pred = model.predict(test)
final=pd.DataFrame()
final['Id']=Id
final['Pawpularity']=pred
final.to_csv('submission.csv',index=False)